In [42]:
from PIL import Image
from zipfile import ZipFile
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

# Loading Images from Disk

In [43]:
!rm -rf ../data/good_data
!rm ../data/good_data

rm: ../data/good_data: No such file or directory


In [44]:
data_path = Path('../data/')
data_zips = [data_path / f for f in os.listdir(data_path)]

for i, f in enumerate(data_zips):
    print(f'{i}: {f}')

0: ../data/good_data.zip
1: ../data/dataCounterClockWiseMessy.zip


In [45]:
f_index = 0
data_file = data_zips[0]
print(data_file)

zip_folder_name = 'good_data'
data_extract_location = data_path / zip_folder_name

../data/good_data.zip


In [46]:
with ZipFile(data_file) as z:
    z.extractall(data_extract_location)

In [47]:
inner_folder = os.listdir(data_extract_location)[0];
data_folder_unzip = data_extract_location / inner_folder
os.listdir(data_folder_unzip)

['labels.csv', 'images']

In [48]:
labels = pd.read_csv(data_folder_unzip / 'labels.csv')
labels

,timestamp,forward/backward,left/right
0,1636315836071,0,0
1,1636315836146,0,0
2,1636315836244,0,0
3,1636315836290,0,0
4,1636315836336,0,0
...,...,...,...
9461,1636316210285,0,0
9462,1636316210333,0,0
9463,1636316210368,0,0
9464,1636316210397,0,0


In [49]:
label_encoded = labels['left/right'].values
encoder = LabelBinarizer()
encoder.fit([-1, 0, 1])
label_encoded = encoder.transform(label_encoded)
print(label_encoded[labels['left/right'] == -1])
print(label_encoded[labels['left/right'] == 0])
print(label_encoded[labels['left/right'] == 1])

[[1 0 0]
 [1 0 0]
 [1 0 0]
 ...
 [1 0 0]
 [1 0 0]
 [1 0 0]]
[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 1 0]
 [0 1 0]
 [0 1 0]]
[[0 0 1]
 [0 0 1]
 [0 0 1]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [51]:
import tensorflow.keras.preprocessing as preprocessing

data_path = data_folder_unzip
print(f'{data_path}')
train_ds = preprocessing.image_dataset_from_directory(
    str(data_path),
    labels=list(label_encoded),
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=False,
    validation_split=0.2,
    subset='training',
)

val_ds = preprocessing.image_dataset_from_directory(
    str(data_path),
    labels=list(label_encoded),
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=False,
    validation_split=0.2,
    subset='validation',
)

../data/good_data/data6
Found 9466 files belonging to 1 classes.
Using 7573 files for training.
Found 9466 files belonging to 1 classes.
Using 1893 files for validation.


In [52]:
for i, (data, cur_labels) in enumerate(train_ds):
    print(data.shape, labels.shape)
    print(cur_labels[:2])
    if i == 5:
        break


(32, 256, 256, 3) (9466, 3)
tf.Tensor(
[[0 1 0]
 [0 1 0]], shape=(2, 3), dtype=int64)
(32, 256, 256, 3) (9466, 3)
tf.Tensor(
[[0 1 0]
 [0 1 0]], shape=(2, 3), dtype=int64)
(32, 256, 256, 3) (9466, 3)
tf.Tensor(
[[0 1 0]
 [0 1 0]], shape=(2, 3), dtype=int64)
(32, 256, 256, 3) (9466, 3)
tf.Tensor(
[[1 0 0]
 [1 0 0]], shape=(2, 3), dtype=int64)
(32, 256, 256, 3) (9466, 3)
tf.Tensor(
[[0 1 0]
 [0 1 0]], shape=(2, 3), dtype=int64)
(32, 256, 256, 3) (9466, 3)
tf.Tensor(
[[0 1 0]
 [0 1 0]], shape=(2, 3), dtype=int64)


# Preprocessing 

## Filter labels and images where the car was moving

In [54]:
filtered_index = (labels['forward/backward'] != 0)


RangeIndex(start=0, stop=9466, step=1)

# Dense Neural Network


In [14]:
import tensorflow as tf
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense
import tensorflow.keras as keras

In [18]:
model = Sequential()
model.add(Dense(128, input_shape=(filter_images_flatten.shape[1],)))
model.add(Dense(256))
model.add(Dense(512))
model.add(Dense(256))
model.add(Dense(128))
model.add(Dense(3, activation='softmax'))

In [19]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['loss', 'accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               29491328  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
Total params: 29,820,547
Trainable params: 29,820,547
Non-trainable params: 0
____________________________________________

# Train/Test/Val Split

In [20]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(filter_images_flatten, lr_labels, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
model.fit(x=X_train, y=y_train, batch_size=50, epochs=50)

Epoch 1/50
146/146 [==============================] - 22s 152ms/step - loss: nan - accuracy: 0.1576
Epoch 2/50
146/146 [==============================] - 21s 142ms/step - loss: nan - accuracy: 0.1576
Epoch 3/50
146/146 [==============================] - 22s 154ms/step - loss: nan - accuracy: 0.1576
Epoch 4/50
146/146 [==============================] - 20s 140ms/step - loss: nan - accuracy: 0.1576
Epoch 5/50
146/146 [==============================] - 19s 130ms/step - loss: nan - accuracy: 0.1576
Epoch 6/50
146/146 [==============================] - 20s 135ms/step - loss: nan - accuracy: 0.1576
Epoch 7/50
146/146 [==============================] - 20s 136ms/step - loss: nan - accuracy: 0.1576
Epoch 8/50
146/146 [==============================] - 22s 149ms/step - loss: nan - accuracy: 0.1576
Epoch 9/50
146/146 [==============================] - 24s 163ms/step - loss: nan - accuracy: 0.1576
Epoch 10/50
112/146 [======================>.......] - ETA: 4s - loss: nan - accuracy: 0.1600

KeyboardInterrupt: 